# Electrode State of Charge

This notebook demonstrates some utilities to work with electrode State of Charge (also sometimes called electrode stoichiometry), using the algorithm from Mohtat et al [1]

[1] Mohtat, P. "Towards ..."

In [1]:
import pybamm

## Create and solve model

In [2]:
spm = pybamm.lithium_ion.SPM()
experiment = pybamm.Experiment([
    "Charge at 1C until 4.2V", 
    "Hold at 4.2V until C/50",
    "Discharge at 1C until 3V",
    "Hold at 3V until C/50",
])
parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Mohtat2020)

sim = pybamm.Simulation(spm, experiment=experiment, parameter_values=parameter_values)
sol = sim.solve()
sol.plot([
    "Terminal voltage [V]", 
    "Current [A]", 
    "Negative electrode SOC",
    "Positive electrode SOC",
])

interactive(children=(FloatSlider(value=0.0, description='t', max=2.3205547079887316, step=0.02320554707988731…

## Solve for electrode SOC

Given a total amount of lithium, $n_{Li}$, electrode capacities, $C_n$ and $C_p$, and voltage limits, $V_{min}$ and $V_{max}$, we can solve for the min and max electrode SOCs, $x_0$, $x_{100}$, $y_0$, and $y_{100}$,  and the cell capacity, $C$, using the algorithm from Mohtat et al [1].
First, we find $x_{100}$ and $y_{100}$ using
$$
\begin{align}
n_{Li} &= \frac{3600}{F}(y_{100}C_p + x_{100}C_n),
\\
V_{max} &= U_p(y_{100}) - U_n(x_{100}).
\end{align}
$$
Then, we find $C$ using
$$
\begin{align}
V_{min} &= U_p(y_{0}) - U_n(x_{0})
\\
&= U_p\left(y_{100} + \frac{C}{C_p}\right) - U_n\left(x_{100} - \frac{C}{C_n}\right)
\end{align}
$$
Finally, $x_0$ and $y_0$ are simply defined as
$$
\begin{align}
x_0 &= x_{100} - \frac{C}{C_n},
\\
y_0 &= y_{100} + \frac{C}{C_p}.
\end{align}
$$
We implement this in pybamm as an algebraic model.

In [9]:
param = pybamm.LithiumIonParameters()

Vmin = 3
Vmax = 4.2
Cn = parameter_values.evaluate(param.C_n_init)
Cp = parameter_values.evaluate(param.C_p_init)
n_Li = parameter_values.evaluate(param.n_Li_init)

Un = param.U_n
Up = param.U_p

x = pybamm.Variable("x")
y = pybamm.Variable("y")
C = pybamm.Variable("C")

ValueError: symbol must evaluate to a constant scalar

In [10]:
parameter_values.process_symbol(param.n_Li_init).render()

+
├── 0.19298869951642003
└── *
    ├── *
    │   ├── *
    │   │   ├── /
    │   │   │   ├── integral dx ['negative electrode', 'separator', 'positive electrode']
    │   │   │   │   └── concatenation
    │   │   │   │       ├── broadcast
    │   │   │   │       │   └── Negative electrode porosity = 0.3
    │   │   │   │       ├── broadcast
    │   │   │   │       │   └── Separator porosity = 0.4
    │   │   │   │       └── broadcast
    │   │   │   │           └── Positive electrode porosity = 0.3
    │   │   │   └── integral dx ['negative electrode', 'separator', 'positive electrode']
    │   │   │       └── broadcast
    │   │   │           └── 1.0
    │   │   └── Typical electrolyte concentration [mol.m-3] = 1000.0
    │   └── 0.000141
    └── Electrode width [m] = 0.205


In [15]:
model = pybamm.BaseModel()
parameter_values.process_symbol(Un(x,0)).render()

/
├── -
│   ├── -
│   │   ├── -
│   │   │   ├── -
│   │   │   │   ├── -
│   │   │   │   │   ├── -
│   │   │   │   │   │   ├── -
│   │   │   │   │   │   │   ├── +
│   │   │   │   │   │   │   │   ├── 0.063
│   │   │   │   │   │   │   │   └── *
│   │   │   │   │   │   │   │       ├── 0.8
│   │   │   │   │   │   │   │       └── function (exp)
│   │   │   │   │   │   │   │           └── *
│   │   │   │   │   │   │   │               ├── -75.0
│   │   │   │   │   │   │   │               └── +
│   │   │   │   │   │   │   │                   ├── x
│   │   │   │   │   │   │   │                   └── 0.007
│   │   │   │   │   │   │   └── *
│   │   │   │   │   │   │       ├── 0.012
│   │   │   │   │   │   │       └── function (tanh)
│   │   │   │   │   │   │           └── /
│   │   │   │   │   │   │               ├── -
│   │   │   │   │   │   │               │   ├── x
│   │   │   │   │   │   │               │   └── 0.127
│   │   │   │   │   │   │               └── 0.016
│   │   │   │   │   │   └──

In [ ]:
parameter_values.process_symbol(pybamm.x_average(param.epsilon_n))

In [ ]:
sim._disc

In [ ]:
c_s = spm.variables["X-averaged negative particle concentration"]
import casadi
y = casadi.MX.sym("y", 31)
sim._disc.process_symbol(pybamm.Multiplication(c_s, 2*pybamm.ones_like(c_s))).to_casadi(y=y)